In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

!pip install plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots


import numpy as np
from scipy.stats import shapiro, normaltest, probplot

from statsmodels.tsa.seasonal import seasonal_decompose

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
%run "variables.py"
%run "functions.ipynb"

In [6]:
# df = pd.read_csv(f'{DATA_DIR}/pred_maint_timeseries_with_failures.csv')
df = pd.read_csv(f'{DATA_DIR}/pred_maint_year_outliers_nulls.csv')
display(df.head())

,machineid,datetimestamp,enginegastemperature1,enginegastemperature2,enginegastemperature3,enginegastemperature4,enginegastemperature5,enginegastemperature6,pressure
0,1,2025-01-01 00:00:00,302.483571,385.408617,303.238443,307.615149,298.829233,298.829315,103.158426
1,1,2025-01-01 00:10:00,303.837174,300.111109,297.861035,297.340913,299.412622,301.110395,98.464047
2,1,2025-01-01 00:20:00,299.877724,301.777757,302.085056,304.162309,298.533004,299.850807,100.190252
3,1,2025-01-01 00:30:00,303.323272,303.895963,294.494511,305.651141,301.865595,298.067635,97.682460
4,1,2025-01-01 00:40:00,297.766293,NaN,301.614999,NaN,289.108329,294.780518,100.345387


In [7]:

df.drop(columns=['machineid'])
df = df.set_index('datetimestamp')
display(df.head())

,machineid,enginegastemperature1,enginegastemperature2,enginegastemperature3,enginegastemperature4,enginegastemperature5,enginegastemperature6,pressure
datetimestamp,,,,,,,,
2025-01-01 00:00:00,1,302.483571,385.408617,303.238443,307.615149,298.829233,298.829315,103.158426
2025-01-01 00:10:00,1,303.837174,300.111109,297.861035,297.340913,299.412622,301.110395,98.464047
2025-01-01 00:20:00,1,299.877724,301.777757,302.085056,304.162309,298.533004,299.850807,100.190252
2025-01-01 00:30:00,1,303.323272,303.895963,294.494511,305.651141,301.865595,298.067635,97.682460
2025-01-01 00:40:00,1,297.766293,NaN,301.614999,NaN,289.108329,294.780518,100.345387


# Features

Create features to perform One-class SVM model in time series dataframe df for anomaly detection

In [13]:
"""from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest

# Create rolling window features for anomaly detection
window_sizes = [3, 5, 10, 20]

# Initialize feature dataframe
df_features = df.copy()

# Create mean_temperature column from temperature sensors
temp_columns = ['enginegastemperature1', 'enginegastemperature2', 'enginegastemperature3', 
                'enginegastemperature4', 'enginegastemperature5', 'enginegastemperature6']
df_features['mean_temperature'] = df_features[temp_columns].mean(axis=1)

# Remove any missing values first
df_features = df_features.dropna()

#
# 5. Additional statistical features
df_features['abs_diff_from_mean'] = abs(df['mean_temperature'] - df['mean_temperature'].mean())
df_features['squared_diff_from_mean'] = (df['mean_temperature'] - df['mean_temperature'].mean()) ** 2

# Create Z-score based anomaly flags for comparison
df_with_zscore_features = df_features.copy()
df_with_zscore_features['max_abs_zscore'] = df_features[['zscore_10', 'zscore_20', 'zscore_50']].abs().max(axis=1)
df_with_zscore_features['anomaly_flag_2sigma'] = (df_with_zscore_features['max_abs_zscore'] > 2).astype(int)
df_with_zscore_features['anomaly_flag_3sigma'] = (df_with_zscore_features['max_abs_zscore'] > 3).astype(int)

# Select features for Isolation Forest model
isolation_features = [col for col in df_features.columns if col not in ['mean_temperature']]
all_isolation_features = [col for col in isolation_features if not col.startswith('anomaly_flag')]

# Remove rows with NaN values (due to rolling windows and lags)
df_isolation_features = df_features[all_isolation_features].dropna()

print(f"Created {len(all_isolation_features)} features for anomaly detection:")
print(f"Features: {all_isolation_features}")
print(f"Feature matrix shape: {df_isolation_features.shape}")
print(f"Data range: {df_isolation_features.index.min()} to {df_isolation_features.index.max()}")

# Display feature statistics
display(df_isolation_features.describe())
"""

'from sklearn.preprocessing import StandardScaler\nfrom sklearn.ensemble import IsolationForest\n\n# Create rolling window features for anomaly detection\nwindow_sizes = [3, 5, 10, 20]\n\n# Initialize feature dataframe\ndf_features = df.copy()\n\n# Create mean_temperature column from temperature sensors\ntemp_columns = [\'enginegastemperature1\', \'enginegastemperature2\', \'enginegastemperature3\', \n                \'enginegastemperature4\', \'enginegastemperature5\', \'enginegastemperature6\']\ndf_features[\'mean_temperature\'] = df_features[temp_columns].mean(axis=1)\n\n# Remove any missing values first\ndf_features = df_features.dropna()\n\n#\n# 5. Additional statistical features\ndf_features[\'abs_diff_from_mean\'] = abs(df[\'mean_temperature\'] - df[\'mean_temperature\'].mean())\ndf_features[\'squared_diff_from_mean\'] = (df[\'mean_temperature\'] - df[\'mean_temperature\'].mean()) ** 2\n\n# Create Z-score based anomaly flags for comparison\ndf_with_zscore_features = df_features.

In [ ]:
## For SVM

In [15]:
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler

# Prepare features for One-Class SVM
# Use the same features we created for Isolation Forest
svm_features = df_isolation_features.copy()

# Scale the features for SVM (important for SVM performance)
scaler = StandardScaler()
svm_features_scaled = scaler.fit_transform(svm_features)

# Train One-Class SVM model
# nu parameter controls the fraction of outliers (typically between 0.01 and 0.1)
# gamma controls the influence of each training example
svm_model = OneClassSVM(nu=0.05, gamma='scale', kernel='rbf')
svm_predictions = svm_model.fit_predict(svm_features_scaled)

# Create anomaly flags (-1 means anomaly, 1 means normal)
svm_anomalies = (svm_predictions == -1).astype(int)

# Add SVM results to dataframe
df_svm_results = df_isolation_features.copy()
df_svm_results['svm_anomaly'] = svm_anomalies
df_svm_results['svm_prediction'] = svm_predictions

print(f"One-Class SVM Results:")
print(f"Total data points: {len(svm_predictions)}")
print(f"Anomalies detected: {sum(svm_anomalies)} ({sum(svm_anomalies)/len(svm_predictions)*100:.2f}%)")
print(f"Normal points: {sum(svm_predictions == 1)} ({sum(svm_predictions == 1)/len(svm_predictions)*100:.2f}%)")

# Display first few results
display(df_svm_results[['svm_anomaly', 'svm_prediction']].head(10))

# Show some statistics about anomalies over time
anomaly_by_month = df_svm_results.groupby(df_svm_results.index.month)['svm_anomaly'].agg(['count', 'sum', 'mean'])
anomaly_by_month.columns = ['total_points', 'anomalies', 'anomaly_rate']
print("\nAnomaly distribution by month:")
display(anomaly_by_month)

Created 26 features for anomaly detection:
Features: ['rolling_mean_3', 'rolling_std_3', 'rolling_min_3', 'rolling_max_3', 'rolling_mean_5', 'rolling_std_5', 'rolling_min_5', 'rolling_max_5', 'rolling_mean_10', 'rolling_std_10', 'rolling_min_10', 'rolling_max_10', 'rolling_mean_20', 'rolling_std_20', 'rolling_min_20', 'rolling_max_20', 'lag_1', 'lag_2', 'lag_3', 'lag_5', 'lag_10', 'zscore_10', 'zscore_20', 'zscore_50', 'abs_diff_from_mean', 'squared_diff_from_mean']
Feature matrix shape: (170669, 26)
One-Class SVM Results:
Total data points: 170669
Anomalies detected: 8534 (5.00%)
Normal points: 162135 (95.00%)


,svm_anomaly,svm_prediction
datetimestamp,,
2025-01-01 12:30:00,0,1
2025-01-01 12:40:00,0,1
2025-01-01 13:00:00,0,1
2025-01-01 13:30:00,0,1
2025-01-01 13:40:00,0,1
2025-01-01 13:50:00,0,1
2025-01-01 14:00:00,0,1
2025-01-01 14:10:00,0,1
2025-01-01 14:20:00,0,1


AttributeError: 'Index' object has no attribute 'month'

# Model

using the features created above, we can now train an one-class SVM model to detect anomalies in the time series data. Show the confusion matrix of the anomalies detected by the Z score model and the defined failure flags for temperature sensors. Add evaluation of the model.